In [2]:
import chromadb

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PDFMinerLoader

In [4]:
load_dotenv()

True

In [ ]:
# llm= ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.7)
GOOGLE_API_KEY='enter your api key'
llm= ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.7, api_key=GOOGLE_API_KEY)

In [ ]:
loader=PDFMinerLoader('The Ultimate Python Handbook.pdf') ### Enter your document path
load_pdf=loader.load()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [7]:
load_pdf

[Document(metadata={'producer': '3.0.10 (5.0.18)', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-16T02:12:11+05:30', 'author': 'Harry Codes', 'moddate': '2024-06-15T22:44:10+02:00', 'total_pages': 61, 'source': 'The Ultimate Python Handbook.pdf'}, page_content='\n\x0cPREFACE \n\nWelcome to the “Ultimate Python Programming Handbook," your comprehensive guide to \nmastering Python programming. This handbook is designed for beginners and anyone looking to \nstrengthen their foundational knowledge of Python, a versatile and user-friendly programming \nlanguage. \n\nPURPOSE AND AUDIENCE \n\nThis handbook aims to make programming accessible and enjoyable for everyone. Whether \nyou\'re a student new to coding, a professional seeking to enhance your skills, or an enthusiast \nexploring Python, this handbook will definitely be helpful. Python\'s simplicity and readability \nmake it an ideal starting point for anyone interested in programming. \n\nSTRUCTURE AND CONTEN

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text= text_splitter.split_documents(load_pdf)

In [9]:
text

[Document(metadata={'producer': '3.0.10 (5.0.18)', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-16T02:12:11+05:30', 'author': 'Harry Codes', 'moddate': '2024-06-15T22:44:10+02:00', 'total_pages': 61, 'source': 'The Ultimate Python Handbook.pdf'}, page_content='PREFACE \n\nWelcome to the “Ultimate Python Programming Handbook," your comprehensive guide to \nmastering Python programming. This handbook is designed for beginners and anyone looking to \nstrengthen their foundational knowledge of Python, a versatile and user-friendly programming \nlanguage. \n\nPURPOSE AND AUDIENCE'),
 Document(metadata={'producer': '3.0.10 (5.0.18)', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-16T02:12:11+05:30', 'author': 'Harry Codes', 'moddate': '2024-06-15T22:44:10+02:00', 'total_pages': 61, 'source': 'The Ultimate Python Handbook.pdf'}, page_content="PURPOSE AND AUDIENCE \n\nThis handbook aims to make programming accessible and enjoyable for every

In [10]:
len(text)

164

In [ ]:
text[2]

In [ ]:
from langchain import embeddings
persist_directory='db'
embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

vector_db= Chroma.from_documents(documents=text,
                                 embedding=embedding,
                                 persist_directory=persist_directory
                                 )

In [1]:
vector_db.persist()
vector_db=None

NameError: name 'vector_db' is not defined

In [ ]:
vector_db=Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
retrive=vector_db.as_retriever(search_kwargs={"k":2})

In [ ]:
retrive.search_type

In [ ]:
docs=retrive.get_relevant_documents('What is python')

In [ ]:
docs

In [ ]:
len(docs)

In [ ]:
from langchain.chains import RetrievalQA
chain=RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retrive, return_source_documents=True)

In [ ]:
query="what is python"
response=chain(query)
response
response['result']

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query=" give me a personal details of jay patel"
response=chain(query)
process_llm_response(response)

In [ ]:
import sys

user_input = input("enter your query")
while True:
  if user_input.lower() in ["exit", "q", "e"]:
    print("exit")
    sys.exit() # Move sys.exit() here
  if user_input != '': # Check if user input is not empty
    query = user_input # Correctly assign user_input to query
    response = chain(query)
    process_llm_response(response)
  user_input = input("enter your query") # Get next input at the end of the loop